In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [62]:
data = pd.read_csv('https://lms.skillfactory.ru/assets/courseware/v1/1d68c628a08cd5521ae70e976577568b/asset-v1:Skillfactory+DST-PRO+15APR2020+type@asset+block/movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
854,tt1007029,13000000,114956699,The Iron Lady,Meryl Streep|Anthony Stewart Head|Harry Lloyd|...,Phyllida Lloyd,Never compromise,"A look at the life of Margaret Thatcher, the f...",105,History|Drama,UK Film Council|Canal+|DJ Films|CinÃ©CinÃ©ma|P...,12/30/2011,6.2,2011
1770,tt0815244,16500000,13620075,Sydney White,Amanda Bynes|Sara Paxton|Matt Long|Jeremy Howa...,Joe Nussbaum,Freshman year is no fairytale,A modern retelling of Snow White set against s...,108,Comedy,Morgan Creek Productions|Clifford Werber Produ...,9/21/2007,5.9,2007
1485,tt0443543,40000000,86915017,The Illusionist,Edward Norton|Jessica Biel|Paul Giamatti|Rufus...,Neil Burger,Nothing Is What It Seems.,"With his eye on a lovely aristocrat, a gifted ...",110,Fantasy|Drama|Thriller|Romance,Michael London Productions|Stillking Films|Bob...,8/18/2006,7.0,2006
1595,tt0265208,25000000,30411183,The Girl Next Door,Emile Hirsch|Elisha Cuthbert|Timothy Olyphant|...,Luke Greenfield,Matt never saw her coming... but all his frien...,Exceptionally ambitious high schooler Matthew ...,108,Comedy,Regency Enterprises|Epsilon Motion Pictures|Ne...,4/9/2004,6.2,2004
1591,tt0375679,6500000,98410061,Crash,Sandra Bullock|Don Cheadle|Matt Dillon|Michael...,Paul Haggis,You think you know who you are. You have no idea.,Los Angeles citizens with vastly separate live...,112,Drama,Blackfriars Bridge Films|Yari Film Group|Bob Y...,5/28/2004,7.1,2004


In [4]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 15 columns):
imdb_id                 1889 non-null object
budget                  1889 non-null int64
revenue                 1889 non-null int64
original_title          1889 non-null object
cast                    1889 non-null object
director                1889 non-null object
tagline                 1889 non-null object
overview                1889 non-null object
runtime                 1889 non-null int64
genres                  1889 non-null object
production_companies    1889 non-null object
release_date            1889 non-null object
vote_average            1889 non-null float64
release_year            1889 non-null int64
profit                  1889 non-null int64
dtypes: float64(1), int64(5), object(9)
memory usage: 221.4+ KB


# Предобработка

In [5]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

In [31]:
data['profit'] = data.revenue - data.budget

In [47]:
def count_genres(genres_str, counts_dict):
    for genre in genres_str.split('|'):
        if genre in counts_dict.keys():
            counts_dict[genre] += 1
        else:
            counts_dict[genre] = 1

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [10]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [9]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget == data.budget.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


ВАРИАНТ 2

In [0]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [0]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

In [11]:
data[data.runtime == data.runtime.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
1157,Gods and Generals,tt0279111


# 3. Какой из фильмов самый короткий (в минутах)?





In [12]:
data[data.runtime == data.runtime.min()][['original_title', 'imdb_id']]

,original_title,imdb_id
768,Winnie the Pooh,tt1449283


In [13]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [14]:
data.runtime.mean()

109.6585494970884

In [ ]:
answers['4'] = '2. 110'

# 5. Каково медианное значение длительности фильмов? 

In [15]:
data.runtime.median()

107.0

In [16]:
answers['5'] = '1. 107'

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [24]:
# лучше код получения столбца profit вынести в Предобработку что в начале
data[data.profit == data.profit.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
239,Avatar,tt0499549


In [19]:
answers['6'] = '5. Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [20]:
data[data.profit == data.profit.min()][['original_title', 'imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


In [21]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [22]:
len(data[data.profit > 0])

1478

In [23]:
answers['8'] = '1. 1478'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [26]:
a = data[(data.release_year == 2008)]
a[a.revenue == a.revenue.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
599,The Dark Knight,tt0468569


In [27]:
answers['9'] = '4. The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [33]:
b = data[(data.release_year <= 2014) & (data.release_year >= 2012)]
b[b.profit == b.profit.min()][['original_title', 'imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


In [34]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [44]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
genres = []

for genres_str in data.genres:
    for genre in genres_str.split('|'):
        if genre not in genres:
            genres.append(genre)

counts = dict()
for genre in genres:
    counts[genre] = len(data[data.genres.str.contains(genre)])

max(counts, key=counts.get)

'Drama'

ВАРИАНТ 2

In [48]:
counts = dict()
data.genres.apply(count_genres, counts_dict=counts)
max(counts, key=counts.get)

'Drama'

In [49]:
answers['11'] = '3. Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [50]:
counts = dict()
data[data.profit > 0].genres.apply(count_genres, counts_dict=counts)
max(counts, key=counts.get)

'Drama'

In [51]:
answers['12'] = '1. Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [57]:
data.groupby(['director']).revenue.sum().sort_values(ascending=False).head()

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
Name: revenue, dtype: int64

In [ ]:
answers['13'] = '5. Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [63]:
data[data.genres.str.contains('Action')].director.value_counts().head()

Paul W.S. Anderson    7
Michael Bay           7
Antoine Fuqua         6
Ridley Scott          6
Robert Rodriguez      6
Name: director, dtype: int64

In [65]:
answers['14'] = '1. Ridley Scott'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

# 18. Самый убыточный фильм от Paramount Pictures

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

# 20. Какой самый прибыльный год для студии Warner Bros?

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

# 23. Для какого режиссера зима – самое продуктивное время года? 

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


ВАРИАНТ 2

# Submission

In [0]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '...', '2': '...'}

In [0]:
# и убедиться что ни чего не пропустил)
len(answers)